## Deliverable 3. Create a Travel Itinerary Map.

In [101]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [152]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Auki,SB,80.58,overcast clouds,-8.7676,160.7034,Auki Motel
1,4,Rikitea,PF,76.59,light rain,-23.1203,-134.9692,People ThankYou
2,5,Vila Velha,BR,75.15,mist,-20.3297,-40.2925,Quality Suites Vila Velha
3,11,Atuona,PF,78.64,clear sky,-9.8000,-139.0333,Villa Enata
4,12,Quatre Cocos,MU,78.10,clear sky,-20.2078,57.7625,Sunrise Attitude Hotel


In [143]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
vacation_start = vacation_df.loc[vacation_df['City'] == 'North Myrtle Beach']
vacation_end = vacation_df.loc[vacation_df['City'] == 'North Myrtle Beach']
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'Florida']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'San Patricio']
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'Horn Lake']

In [165]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.

start = tuple(vacation_start[["Lat", "Lng"]].to_numpy()[0])
end = tuple(vacation_end[["Lat", "Lng"]].to_numpy()[0])
stop1 = tuple(vacation_stop1[["Lat", "Lng"]].to_numpy()[0])
stop2 = tuple(vacation_stop2[["Lat", "Lng"]].to_numpy()[0])
stop3 = tuple(vacation_stop3[["Lat", "Lng"]].to_numpy()[0])

start

(33.816, -78.68)

In [149]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
directions = gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3], 
                                    travel_mode='WALKING' or 'DRIVING' or 'BICYCLING')

fig.add_layer(directions)

fig

Figure(layout=FigureLayout(height='420px'))

In [146]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,379,North Myrtle Beach,US,78.03,clear sky,33.8160,-78.6800,Avista Resort
1,264,Florida,US,76.08,clear sky,28.7505,-82.5001,Sportsmens Cove
2,31,San Patricio,US,84.61,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
3,94,Horn Lake,US,76.03,broken clouds,34.9554,-90.0348,"Hilton Garden Inn Memphis/Southaven, MS"


In [162]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [164]:
# 11a. Add a marker layer for each city to the map.
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))